In [1]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_max_distance_to_target_surface, contact_optimization, show_selected_joints, get_fixed_boundary_joint)

import vis.fields
import matplotlib.cm as cm
import time

In [2]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [3]:
rod_length = 0.05979705114046488
width = rod_length / 20 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('lilium_1', thickness, width, weaving_dir)

0.01494926278511622 0.0010464483949581352


In [7]:
import py_newton_optimizer
# Optimization parameters.
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 10
drw = 0.01
dsw = 0.1

In [8]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width = True)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), quaternion=(…

In [9]:
curved_linkage.setExtendedDoFsPSRL(np.load('../../nature_paper_experiments/figure_4/zoo/results/lilium_1/lilium_1_dof.npy'))

In [10]:
curved_linkage.attraction_weight = 1e-5

In [11]:
fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage)

In [12]:
def eqm_callback(prob, i):
    curved_linkage_view.update()

In [13]:
OPTS.niter = 10000

In [14]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)

0	0.000253253	0.274372	0.274372	1	1
1	4.03807e-05	0.0514669	0.0514669	1	1
2	2.18512e-05	0.0128603	0.0128603	1	1
3	1.71845e-05	0.00633637	0.00633637	1	1
4	1.46965e-05	0.00397823	0.00397823	1	1
5	1.25211e-05	0.00274907	0.00274907	1	1
6	1.03527e-05	0.00198238	0.00198238	1	1
7	8.21998e-06	0.00142326	0.00142326	1	1
8	6.30355e-06	0.000980872	0.000980872	1	1
9	4.7983e-06	0.000639965	0.000639965	1	1
10	3.76225e-06	0.000398065	0.000398065	1	1
11	3.09462e-06	0.000241629	0.000241629	1	1
12	2.64906e-06	0.000148517	0.000148517	1	1
13	2.31679e-06	9.53873e-05	9.53873e-05	1	1
14	2.03885e-06	6.58754e-05	6.58754e-05	1	1
15	1.79919e-06	5.42224e-05	5.42224e-05	1	1
16	1.60033e-06	6.5439e-05	6.5439e-05	1	1
17	1.43583e-06	0.000102079	0.000102079	1	1
18	1.29241e-06	0.000153826	0.000153826	1	1
19	1.15974e-06	0.000168555	0.000168555	1	1
20	1.03617e-06	0.000175705	0.000175705	1	1
21	9.3769e-07	0.000174373	0.000174373	1	1
22	8.74151e-07	0.000125301	0.000125301	1	1
23	8.37865e-07	7.90836e-05	7.90836e-05	1	0
24	8.3

In [15]:
import structural_analysis

In [16]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)

In [17]:
cfm[:, 0].argsort()[-1]

130

In [18]:
max(cfm[:, 0])

1.41495557034462e-07

### Deactivate Crossing

In [19]:
import deactivate_crossing_helper
importlib.reload(deactivate_crossing_helper)
from deactivate_crossing_helper import (write_deactivated_linkage, copy_over_under, get_segment_index_from_joints, get_angle_from_kappa, get_kappa_from_angle, no_resample_deactivation, resample_deactivation, get_neighbors)

In [20]:
def create_deactivated_linkage(original_linkage, deactive_index, original_model_name, deactivated_model_name):
    ju = original_linkage.joint(deactive_index)
    nbs = ju.neighbors()
    print('neighbors: ', nbs)
    deactive_vertex_map = [i if i < deactive_index else i - 1 for i in range(original_linkage.numJoints())]
    write_deactivated_linkage(original_model_name, deactivated_model_name, deactive_index, deactive_vertex_map, nbs)
    
    with so(): deactivate_curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = deactivated_model_name, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=elastic_rods.InterleavingType.weaving)
    deactivate_curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    curved_save_tgt_joint_pos = deactivate_curved_linkage.jointPositions();
    deactivate_curved_linkage.attraction_weight = 1e-5

#     Copy the over under assignment from the original model since BFS will not find a perfect over under in the new topology
    copy_over_under(deactivate_curved_linkage, original_linkage, deactive_vertex_map, deactive_index)
    vf = show_selected_joints(deactivate_curved_linkage, [deactive_vertex_map[12], deactive_vertex_map[258]])
    deactivate_curved_linkage.setDoFs(deactivate_curved_linkage.getDoFs())

    resample_deactivation(deactivate_curved_linkage, original_linkage, deactive_vertex_map, nbs, deactive_index, io)
    return deactivate_curved_linkage, deactive_vertex_map

In [21]:
deactive_index = 130

In [22]:
deactivated_model_name = ('/'.join(io.MODEL_PATH.split('/')[:-1] + [io.MODEL_NAME + '_deactive_joint_{}.obj'.format(deactive_index)]))

In [23]:
deactivate_linkage, deactive_vertex_map = create_deactivated_linkage(curved_linkage, deactive_index, io.MODEL_PATH, deactivated_model_name)

neighbors:  [302, 129, 301, 131]
928
929


In [24]:
deactivate_curved_linkage_view = linkage_vis.LinkageViewer(deactivate_linkage)
deactivate_curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), quaternion=(…

In [25]:
topColor =[150/255., 150/255., 150/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = deactivate_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [26]:
deactivate_curved_linkage_view.update(scalarField = colors)

In [27]:
deactivate_fixed_boundary_joints = get_fixed_boundary_joint(deactivate_linkage)

In [28]:
def eqm_callback(prob, i):
    deactivate_curved_linkage_view.update()

In [29]:
elastic_rods.compute_equilibrium(deactivate_linkage, callback = eqm_callback, options = OPTS, fixedVars = deactivate_fixed_boundary_joints)

0	0.00217072	0.408541	0.408541	1	1
1	0.00162877	0.108388	0.108388	1	1
2	0.00144461	0.0567587	0.0567587	1	1
3	0.00139333	0.046977	0.046977	1	1
4	0.00132413	0.0381046	0.0381046	1	1
5	0.00129622	0.0350669	0.0350669	1	1
6	0.00125054	0.0311663	0.0311663	1	1
7	0.00123039	0.0296128	0.0296128	1	1
8	0.00119486	0.027345	0.027345	1	1
9	0.00117858	0.0263636	0.0263636	1	1
10	0.00114897	0.0248146	0.0248146	1	1
11	0.00113514	0.0241265	0.0241265	1	1
12	0.00110929	0.0230471	0.0230471	1	1
13	0.00109693	0.0225693	0.0225693	1	1
14	0.00107314	0.0217998	0.0217998	1	1
15	0.00102979	0.0202731	0.0202731	1	1
16	0.00101095	0.0195204	0.0195204	1	1
17	0.00097622	0.018551	0.018551	1	1
18	0.000959835	0.0181133	0.0181133	1	1
19	0.000928335	0.0174367	0.0174367	1	1
20	0.00091358	0.0170254	0.0170254	1	1
21	0.000886266	0.0162482	0.0162482	1	1
22	0.000841748	0.0191728	0.0191728	1	1
23	0.000816493	0.0151034	0.0151034	1	1
24	0.000776865	0.0138195	0.0138195	1	1
25	0.000758882	0.0133094	0.0133094	1	1
26	0.000726187	0.0126359	

In [30]:
vf = show_selected_joints(deactivate_linkage, [deactive_vertex_map[302], deactive_vertex_map[301], deactive_vertex_map[129], deactive_vertex_map[131]])
deactivate_curved_linkage_view.update(vectorField = vf, scalarField = colors)